In [59]:
import numpy as np
import pandas as pd
import pymysql
import datetime
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import pickle
import matplotlib.pyplot as plt
from constants import * 

In [7]:
try:
    # Connect to the database
    connection = pymysql.connect(host=DB,
                                user=DB_USER,
                                password=DB_PW,
                                database=CITY)
except pymysql.Error as e:
    e

In [8]:
# Close the connection
connection.close()

# Now df contains the data from your table as a DataFrame
print(df.head())


/tmp/ipykernel_16204/2028419593.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


   number  available_bikes  available_bike_stands  last_update
0       1               15                     16   1710576480
1       1               15                     16   1710577085
2       1               16                     15   1710577691
3       1               16                     15   1710578296
4       1               16                     15   1710578901


In [19]:
# make database connection
try:
    # Connect to the database
    connection = pymysql.connect(host=DB,
                                user=DB_USER,
                                password=DB_PW,
                                database=CITY)
except pymysql.Error as e:
    print(e)

In [20]:
# get availability
get_availability = "SELECT * FROM availability"
df_availability = pd.read_sql_query(get_availability, connection)

# get weather
get_weather = "SELECT * FROM weather"
df_weather = pd.read_sql_query(get_weather, connection)

connection.close()

/tmp/ipykernel_16204/1168624268.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_availability = pd.read_sql_query(get_availability, connection)
/tmp/ipykernel_16204/1168624268.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_weather = pd.read_sql_query(get_weather, connection)


In [43]:
df_availability.head()

,number,available_bikes,available_bike_stands,last_update
0,1,15,16,1710576480
1,1,15,16,1710577085
2,1,16,15,1710577691
3,1,16,15,1710578296
4,1,16,15,1710578901


In [44]:
df_weather.head()

,last_update,rain,temp,hum
0,1712652300,0.03,7.0,81.0
1,1712653200,0.11,7.0,81.0
2,1712654100,0.11,8.0,76.0
3,1712655000,0.11,8.0,76.0
4,1712655900,0.11,8.0,76.0


In [45]:
# merge availability with weather data
df = pd.merge_asof(df_availability.sort_values('last_update'), 
                          df_weather.sort_values('last_update'), 
                          on='last_update')

df.head()

,number,available_bikes,available_bike_stands,last_update,rain,temp,hum
0,10,11,5,1710576107,NaN,NaN,NaN
1,95,38,2,1710576111,NaN,NaN,NaN
2,60,14,16,1710576117,NaN,NaN,NaN
3,20,1,29,1710576118,NaN,NaN,NaN
4,105,3,33,1710576118,NaN,NaN,NaN


Unfortunately, the scraper that gathered the weather data was failing for the first few weeks and we weren't aware, so the data we have available isn't as comprehensive as we would like

In [46]:
df = df.dropna()
df.shape

(149328, 7)

The day of the week and the hour of the day are likely useful predictors for bike availability, and as such we want to impute these from the data collected

In [47]:
def get_day_and_hour(timestamp):
    datetime_obj = pd.to_datetime(timestamp, unit='s')
    return datetime_obj.hour, datetime_obj.dayofweek

In [48]:
df['day'], df['hour'] = zip(*df['last_update'].apply(get_day_and_hour))

Additionally, instead of prpedicting both `available_bikes` and `available_parking`, it makes instead to predict for just `availability`, which is effectively a ratio of the available bikes to the overall capacity of the station

This simplifies the analytics pipeline and will make the predictions more intutive: high availability is desirable in a departure station and low availability (high parking) is desirable in an arrival station

In [49]:
df['availability'] = df['available_bikes'] / (df['available_bikes'] + df['available_bike_stands'])

In [50]:
df.head()

,number,available_bikes,available_bike_stands,last_update,rain,temp,hum,day,hour,availability
460665,97,8,32,1712652301,0.03,7.0,81.0,8,1,0.200000
460666,53,19,21,1712652306,0.03,7.0,81.0,8,1,0.475000
460667,58,38,2,1712652313,0.03,7.0,81.0,8,1,0.950000
460668,24,10,10,1712652319,0.03,7.0,81.0,8,1,0.500000
460669,44,2,28,1712652325,0.03,7.0,81.0,8,1,0.066667


We will be making predictions for each station based on `rain`, `temp`, `hum`, `day` and `hour`: `day` is a categorical variable and we will treat the others as continuous

The first stage of our pipeline will be declaring `day` as a categorical variable

It is also best practise to standardise the scale of the continuous features

And finally, to add some non-linearity, we will impute some polynomial features for the continuous features

In [61]:
categorical = ['day']
continuous = ['rain', 'temp', 'hum', 'hour']

preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(), categorical),
        ('scaling', StandardScaler(), continuous),
        ('poly', PolynomialFeatures(degree=2), continuous)
    ],
    remainder='passthrough'
)

## Linear Regression

Let's keep it simple and start with a linear regression

In [57]:
lr_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())  # Add Linear Regression as the final step
])

In [66]:
for station_number in df['number'].unique():
    df_station = df[df['number'] == station_number]
    X_train, X_test, y_train, y_test = train_test_split(df_station[categorical + continuous], df_station['availability'], test_size=0.2, random_state=69)

    print("Station number:", station_number, "R^2 Score:", lr_pipeline.fit(X_train, y_train).score(X_test, y_test))

Station number: 97 R^2 Score: 0.6292366667416187
Station number: 53 R^2 Score: 0.5969795802533711
Station number: 58 R^2 Score: 0.6218622058086514
Station number: 24 R^2 Score: 0.43404677435193617
Station number: 44 R^2 Score: 0.6902700676728959
Station number: 20 R^2 Score: 0.7651677876813636
Station number: 74 R^2 Score: 0.6112123697744438
Station number: 80 R^2 Score: 0.3553049343798558
Station number: 29 R^2 Score: 0.6469902428360788
Station number: 47 R^2 Score: 0.6095194680644069
Station number: 73 R^2 Score: 0.5715964479041684
Station number: 88 R^2 Score: 0.46754362110248415
Station number: 3 R^2 Score: 0.6308924477076583
Station number: 42 R^2 Score: 0.5622654551803173
Station number: 71 R^2 Score: 0.15302434514291263
Station number: 117 R^2 Score: 0.6263216383165278
Station number: 78 R^2 Score: 0.3721581544429252
Station number: 91 R^2 Score: 0.650393498056892
Station number: 22 R^2 Score: 0.4096474223567308
Station number: 33 R^2 Score: 0.5859321337796066
Station number: 62

In [14]:
# for each station
    # train and test split
    # pipe.fit(X_train, y_train).score(X_test, y_test)
    # add resuls to database